In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm


In [3]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths and hyperparameters
DATA_DIR = "dataset"
BATCH_SIZE = 32
NUM_CLASSES = 2
NUM_EPOCHS = 5
LR = 1e-4

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Datasets
train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transforms)
val_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_test_transforms)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=val_test_transforms)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Load pretrained ResNet50
model = models.resnet50(pretrained=True)

# Freeze all layers except the last few
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, NUM_CLASSES)
)

model = model.to(device)

# Load previously saved weights
model.load_state_dict(torch.load("resnet50_deepfake_finetuned.pth", map_location=device))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LR)

# Training loop
def train_model():
    model.train()
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        correct = 0
        total = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

        validate_model()

# Validation loop
def validate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"[VALIDATION ACCURACY] {100 * correct / total:.2f}%")

# Start training from loaded weights
train_model()

# Save updated model
torch.save(model.state_dict(), "resnet50_deepfake_finetuned_continue.pth")

Using device: cuda


C:\Users\samriddhi singh\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\samriddhi singh\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/5 [Train]: 100%|█████████████████████████████████████| 4376/4376 [08:52<00:00,  8.22it/s, acc=85.2, loss=0.222]


[VALIDATION ACCURACY] 84.04%


Epoch 2/5 [Train]: 100%|█████████████████████████████████████| 4376/4376 [08:05<00:00,  9.01it/s, acc=87.4, loss=0.341]


[VALIDATION ACCURACY] 85.34%


...

[VALIDATION ACCURACY] 85.65%


Epoch 4/5 [Train]: 100%|████████████████████████████████████| 4376/4376 [07:58<00:00,  9.14it/s, acc=88.3, loss=0.0109]


[VALIDATION ACCURACY] 84.71%


Epoch 5/5 [Train]: 100%|███████████████████████████████████| 4376/4376 [07:57<00:00,  9.16it/s, acc=88.6, loss=0.00216]


[VALIDATION ACCURACY] 84.57%


In [5]:
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"[TEST ACCURACY] {100 * correct / total:.2f}%")

test_model()


[TEST ACCURACY] 69.37%


In [ ]:
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)
train_model()  # Train again
